# EGCI 305: Chapter 3 (Continuous Distributions)

Outline
> 1. [Packages](#ch3_packages)

> 2. [Integral calculation](#ch3_integral)
>    - [Example: dynamic load](#ch3_ex_dynamic)

> 3. [Uniform distribution](#ch3_uniform)
>    - [Example: tire](#ch3_ex_tire)

> 4. [Normal distribution](#ch3_normal)
>    - [Example: Z distribution](#ch3_ex_z)
>    - [Example: rear-end collision](#ch3_ex_rear)
>    - [Example: water overflow](#ch3_ex_water)

> 5. [Exponential distribution](#ch3_exponential)
>    - [Example: stress range](#ch3_ex_stress)

Functions
> - [getPercentileValues](#ch3_func_percentile)
> - [X and Z conversions](#ch3_func_xz)

<a name="ch3_packages"></a>

## Packages
> - **numpy** -- to work with array manipulation
> - **matplotlib** -- to work with visualization (backend)
> - **seaborn** -- to work with high-level visualization
> - **scipy.stats** -- to work with stat
> - **sympy** -- to work with integral calculation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Numpy version =", np.version.version)
print("Seaborn version =", sns.__version__)

import scipy
print("Scipy version =", scipy.__version__)

from scipy import stats
from scipy.stats import uniform         # Uniform distribution
from scipy.stats import norm            # Normal distribution
from scipy.stats import expon           # Exponential distribution


import sympy
print("Sympy version =", sympy.__version__)

from sympy import *

<a name="ch3_integral"></a>

## Integral Calculation
- **[Manual:sympy.integrals.integrals.Integral](https://docs.sympy.org/latest/modules/integrals/integrals.html#sympy.integrals.integrals.Integral)** 

<a name="ch3_ex_dynamic"></a>

### Example : dynamic load

In [ ]:
#----- Declare variable & function symbols
x  = Symbol('x')
y  = Symbol('y')
fy = Function('fy')(y)
fy = (1/8) + (3*y/8)

#------ fy by dy 
cdf_x_nobound = Integral(fy, y)            
cdf_x_bound   = Integral(fy, (y, 0, x))    

pprint(cdf_x_bound, use_unicode=True)       
cdf_x_bound.doit() 

In [ ]:
#------ Substitute values in the solved integral
Fx = cdf_x_bound.doit()
upper = Fx.subs(x, 1.5)
lower = Fx.subs(x, 1)
print("F(1.5) - F(1) = %.4f" % (upper-lower))

#------ Or solve integral using 1-1.5 bounds
result = Integral(fy, (y, 1, 1.5)).doit()
print("Result = %.4f" % result)

<a name="ch3_uniform"></a>

## Uniform Distribution
- **[Manual: scipy.stats.rv_continuous](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html#scipy.stats.rv_continuous)**
- **[Manual: scipy.stats.uniform](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html#scipy.stats.uniform)**
    > - For U(A, B) --> loc = A, scale = B-A
    > - Default loc = 0
    > - Defalut scale = 1

<a name="ch3_ex_tire"></a>

### Example : tire
> X $\sim$ U(0, 360)

<a name="ch3_func_percentile"></a>
#### Function : getPercentileValues

In [ ]:
# Get values at percentiles 1-99 of a certain distribution

def getPercentileValues(dist, loc, scale):
    r_values = np.linspace( dist.ppf(0.01, loc, scale),
                            dist.ppf(0.99, loc, scale),
                            100
                          )
    return r_values

In [ ]:
#------ Uniform params
A = 0          
B = 360 

loc   = A
scale = B-A


#------ Some float values
r_values = np.arange(-100, 500, 1)                      # step by 1
# r_values = np.linspace(-100, 500, 100)                # 100 values of equal interval
# r_values = getPercentileValues(uniform, loc, scale)   # percentiles 1-99

#------ pdf & cdf of each value
pdf = uniform.pdf(r_values, loc, scale)
cdf = uniform.cdf(r_values, loc, scale)

#------ mean, variance, sd
mean = uniform.mean(loc, scale)
var  = uniform.var(loc, scale)
std  = uniform.std(loc, scale)
print("mean = %.2f, variance = %.2f, sd = %.2f \n" % (mean, var, std))

#------ pdf, cdf plots
fig = plt.figure( figsize = (12,4) )
plt.subplot(121)
plt.plot(r_values, pdf)
plt.title("pdf of U(%d, %d)" % (A, B))

plt.subplot(122)
plt.plot(r_values, cdf)
plt.title("cdf of U(%d, %d)" % (A, B))
plt.show()

**Questions**
> - Q1 : P(90 <= X <= 180) 
> - Q2 : P(0 <= X <= 90) + P(270 <= X <= 360)

In [ ]:
cdf_90  = uniform.cdf(90, loc, scale)
cdf_180 = uniform.cdf(180, loc, scale)
Q1 = cdf_180 - cdf_90
print("P(X <= 90)  = %.3f" % cdf_90)
print("P(X <= 180) = %.3f" % cdf_180)
print("Q1 = %.3f \n" % Q1)

cdf_0   = uniform.cdf(0, loc, scale)
cdf_270 = uniform.cdf(270, loc, scale)
cdf_360 = uniform.cdf(360, loc, scale)
Q2 = (cdf_90 - cdf_0) + (cdf_360 - cdf_270)
print("P(X <= 0)   = %.3f" % cdf_0)
print("P(X <= 90)  = %.3f" % cdf_90)
print("P(X <= 270) = %.3f" % cdf_270)
print("P(X <= 360) = %.3f" % cdf_360)
print("Q2 = %.3f \n" % Q2)

<a name="ch3_normal"></a>

## Normal Distribution
- **[Manual: scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm)**
    > - For N($\mu$, $\sigma$<sup>2</sup>) --> loc = $\mu$, scale = $\sigma$
    > - Default loc = 0
    > - Default scale = 1

<a name="ch3_ex_z"></a>

### Example : Z distribution
> Z $\sim$ N(0, 1)

**Questions**
> - Q1 : P(Z <= 1.25)
> - Q2 : P(Z <= -1.25)
> - Q3 : P(-0.38 <= Z <= 1.25)
> - Q4 : P(Z < ?) = 0.05
> - Q5 : P(Z < ?) = 0.95

In [ ]:
#------ Z params
loc   = 0
scale = 1


#------ Find probability (i.e. cdf) from Z value
Q1 = norm.cdf(1.25)
print("Q1 = %.4f" % Q1)

Q2 = norm.cdf(-1.25)
print("Q2 = %.4f" % Q2)

Q3 = norm.cdf(1.25) - norm.cdf(-0.38)
print("Q3 = %.4f \n" % Q3)


#------ Find Z value (i.e. percentile) from cdf
Q4 = norm.ppf(0.05)
print("Q4 = %.4f" % Q4)

Q5 = norm.ppf(0.95)
print("Q5 = %.4f" % Q5)

<a name="ch3_ex_rear"></a>

### Example : rear-end collision
> - $\mu$ = 1.25, $\sigma$ = 0.46
> - X $\sim$ N(1.25, 0.46 * 0.46)

**Questions**
> - Q1 : P(1 < X < 1.75)
> - Q2 : P(X > 2)

<a name="ch3_func_xz"></a>
#### Functions : X and Z conversions

In [ ]:
# Convert between X and Z of normal distribution

def getZfromX(x, loc, scale):
    z = (x - loc)/scale
    return z

def getXfromZ(z, loc, scale):
    x = z * scale + loc
    return x

In [ ]:
#------ With programming, we can use X directly
loc    = 1.25
scale  = 0.46
xlower = 1
xupper = 1.75
Q1_x = norm.cdf(xupper, loc, scale) - norm.cdf(xlower, loc, scale)
print("Q1 by X = %.4f" % Q1_x)

#------ But with manual calculation, we have to use Z
#       For manual Z-table lookup, use Z with 2 decimals
zlower = getZfromX(xlower, loc, scale)
zupper = getZfromX(xupper, loc, scale)
# zlower = round(zlower, 2)
# zupper = round(zupper, 2)
Q1_z = norm.cdf(zupper) - norm.cdf(zlower)
print("Q1 by Z = %.4f, zlower = %.2f, zupper = %.2f" % 
      (Q1_z, zlower, zupper)
     )

In [ ]:
x = 2
Q2_x = 1 - norm.cdf(x, loc, scale)

z = getZfromX(x, loc, scale)
Q2_z = 1 - norm.cdf(z)

print("Q2 by X = %.4f" % Q2_x)
print("Q2 by Z = %.4f, z = %.2f" % (Q2_z, z))

<a name="ch3_ex_water"></a>

### Example : water overflow
> - $\mu$ = 64, $\sigma$ = 0.78
> - X $\sim$ N(64, 0.78 * 0.78)

**Questions**
> - Q1 : P(X < ?) = 0.995

In [ ]:
loc   = 64
scale = 0.78
Q1    = norm.ppf(0.995, loc, scale)
print("Q1 = %.4f" % Q1)

z = norm.ppf(0.995)
x = getXfromZ(z, loc, scale)
print(" z = %.4f \n x = %.4f \n" % (z, x))


#------ Values at percentiles 1-99 for pdf plot
x_values = getPercentileValues(norm, loc, scale)
z_values = getPercentileValues(norm, 0, 1)

fig = plt.figure( figsize = (12,4) )
plt.subplot(121)
plt.plot(x_values, norm.pdf(x_values, loc, scale))
plt.title("pdf of X")

plt.subplot(122)
plt.plot(z_values, norm.pdf(z_values))
plt.title("pdf of Z")
plt.show()

<a name="ch3_exponential"></a>

## Exponential Distribution
- **[Manual: scipy.stats.expon](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html)**
    > - For Exp($\lambda$) --> scale = 1/$\lambda$
    > - Default loc = 0
    > - Default scale = 1

<a name="ch3_ex_stress"></a>

### Example : stress range
> - 1 / $\lambda$ = 6
> - X $\sim$ Exp(1/6)

**Questions**
> - Q1 : P(X <= 10)
> - Q2 : P(5 <= X <= 10)

In [ ]:
#------ Exponential params
loc   = 0
scale = 6
lmd   = 1/scale                         # lambda is Python's keyword
print("lambda = %.4f" % lmd)

#------ mean, variance, std
mean = expon.mean(loc, scale)
var  = expon.var(loc, scale)
std  = expon.std(loc, scale)
print("mean = %.2f, variance = %.2f, sd = %.2f \n" % (mean, var, std))

Q1 = expon.cdf(10, loc, scale)
Q2 = expon.cdf(10, loc, scale) - expon.cdf(5, loc, scale)
print("Q1 = %.4f" % Q1)
print("Q2 = %.4f" % Q2, "\n")


r_values = getPercentileValues(expon, loc, scale)
fig = plt.figure( figsize = (12,4) )
plt.subplot(121)
plt.plot(r_values, expon.pdf(r_values, loc, scale))
plt.title("pdf of Exp(%.4f)" % lmd)

plt.subplot(122)
plt.plot(r_values, expon.cdf(r_values, loc, scale))
plt.title("cdf of Exp(%.4f)" % lmd)
plt.show()